In [93]:
#Import lib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import time

import torch
from torchtext import data, datasets, vocab
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from pythainlp.tokenize import word_tokenize

In [94]:
#ไฟล์ Dataset

word_file = pd.read_csv("รวมคำ.csv")

In [95]:
# ใน Dataset ประกอบด้วย 2 ส่วน คือคำหรือประโยค กับ Label โดย Label จะแบ่งได้ดังนี้

"""
1 = Positive
2 = High Positive
3 = Natural
4 = Negative
5 = High Negative
"""

word_file.head()

,ดี,1
0,ขอบคุณ,1
1,สู้เสมอ,1
2,ผ่าน,1
3,โคตรสนุก,1
4,สุดยอด,1


In [96]:
#แปลง DataFrame ให้เป็น List เพื่อเอาไปทำ Tokenize

word_list = word_file.values.tolist()

print(word_list[0])

['ขอบคุณ', 1]


In [97]:
# ทำ Tokenize ด้วย Pythainlp

new_word_list = []

for a_list in word_list:
    new_list = word_tokenize(a_list[0])
    label_list = [new_list, a_list[1]]
    new_word_list.append(label_list)

print(new_word_list)

, [['สาน', 'ฝัน', 'ให้', 'จริง'], 1], [['อาสา'], 1], [['อยาก', 'กอด'], 1], [['เตือนสติ'], 1], [['สันติสุข'], 1], [['สำนึก'], 1], [['เลือก', 'ตามใจ'], 1], [['โค', 'รต', 'น่ารัก'], 1], [['มหัศจรรย์'], 1], [['สิ่ง', 'พิเศษ'], 1], [['วิเศษ'], 1], [['ผ่อนคลาย'], 1], [['ยอมรับ'], 1], [['เจริญรุ่งเรือง'], 1], [['เสียสละ'], 1], [['ไถ่โทษ'], 1], [['ไถ่บาป'], 1], [['อบอวล'], 1], [['สะอาด'], 1], [['เอาใจใส่'], 1], [['ภาวนา'], 1], [['ฟุ้งเฟื่อง'], 1], [['เกียรติคุณ'], 1], [['ด้าน', 'บวก'], 1], [['ปลื้ม', 'ดีใจ'], 1], [['คึกคัก'], 1], [['เจตนา', 'ดี'], 1], [['จิต', 'แจ่มใส'], 1], [['เปรี้ยงปร้าง'], 1], [['ฮิต'], 1], [['ที่', 'นิยม'], 1], [['ปลอบใจ'], 1], [['ปรองดอง'], 1], [['ไมตรี'], 1], [['มารยาท'], 1], [['รอบคอบ'], 1], [['บริสุทธิ์ใจ'], 1], [['เคารพนับถือ'], 1], [['วุฒิภาวะ'], 1], [['สุขุม'], 1], [['ฟื้นฟู'], 1], [['ล้ำสมัย'], 1], [['ดี', 'ที่สุด'], 1], [['ทันเวลา', 'พอดี'], 1], [['สุจริต'], 1], [['น่าเชื่อถือ'], 1], [['เท่ห์'], 1], [['ไว้วางใจ'], 1], [['เสน่ห์'], 1], [['ทรงพลัง'], 1], [['ฮัก'], 

In [99]:
# เราจะแปลง คำให้เป็น Tensor จึงต้องเอาคำซ้ำออก โดยเปลี่ยนให้เป็น Set

word_set = set()

for a_list in new_word_list:
    word_set.update(a_list[0])
        
print(word_set)

{'สาปแช่ง', 'แปะ', 'บ่น', 'เครียด', 'ขี้เกียจ', 'หมดอายุ', 'อี', 'ป', 'ก่อน', 'ก็ได้', 'เศร้าสลด', 'เหมือนกัน', 'มัว', 'พึ่งพิง', 'อยู่', 'ต่อสู้', 'กาลเวลา', 'ย่อยยับ', 'ส', 'คลาดเคลื่อน', 'อัปยศ', 'สัญญา', 'ไข้หวัด', 'ชนะ', 'ช่วยเหลือ', 'ประการ', 'ศก', 'พลัง', 'ได้', 'การกุศล', 'หรือ', 'ตัวเอง', 'เลือก', 'ถลอก', 'ตอแหล', 'ขอให้', 'ร่างกาย', 'ระดับ', 'ทัน', 'ชื่นชม', 'แฮปปี้', 'ตายห่า', 'ยืดหยุ่น', 'ทำไม', 'ล้อเลียน', 'สังหาร', 'พา', 'คิดถึง', 'มองโลก', 'เสร็จ', 'เพลิน', 'ประทับใจ', 'เปิดเผย', 'นิ่ง', 'ล้ม', 'หัว', 'เชื่อมั่น', 'มากกว่า', 'พราก', 'จอมปลอม', 'พังยับเยิน', 'สำนึก', 'ไมตรี', 'ซ้ำร้าย', 'ริดรอน', 'ชำระ', 'หล่อ', 'เเม่ง', 'ความสามัคคี', 'กุศล', 'หมั่น', 'ง่าย', 'จากไป', 'ใจ', 'กำ', 'น่าสน', 'รู้', 'มิตรภาพ', 'คลี่คลาย', 'จม', 'กบฏ', 'เจ็บแสบ', 'ดวงดี', 'เพื่อนสนิท', 'ย่อท้อ', 'เสียความรู้สึก', 'ผู้ชาย', 'สะดวก', 'สานต่อ', 'ประชาธิปไตย', 'เตก', 'เวลา', 'เจ้าชู้', 'ความเหลื่อมล้ำ', 'ฟุ่มเฟือย', 'ลม', 'พอเพียง', 'ก้อนหิน', 'สิ่งแวดล้อม', 'มะเร็ง', 'ความหวังดี', 'หกล้ม', 'ว่ะ'

In [100]:
#Set เอาคำซ้อออกหมดแล้ว เอากลับไปเป็น List อีกครั้ง

word_set_to_list = list(word_set)

print(word_set_to_list)

['สาปแช่ง', 'แปะ', 'บ่น', 'เครียด', 'ขี้เกียจ', 'หมดอายุ', 'อี', 'ป', 'ก่อน', 'ก็ได้', 'เศร้าสลด', 'เหมือนกัน', 'มัว', 'พึ่งพิง', 'อยู่', 'ต่อสู้', 'กาลเวลา', 'ย่อยยับ', 'ส', 'คลาดเคลื่อน', 'อัปยศ', 'สัญญา', 'ไข้หวัด', 'ชนะ', 'ช่วยเหลือ', 'ประการ', 'ศก', 'พลัง', 'ได้', 'การกุศล', 'หรือ', 'ตัวเอง', 'เลือก', 'ถลอก', 'ตอแหล', 'ขอให้', 'ร่างกาย', 'ระดับ', 'ทัน', 'ชื่นชม', 'แฮปปี้', 'ตายห่า', 'ยืดหยุ่น', 'ทำไม', 'ล้อเลียน', 'สังหาร', 'พา', 'คิดถึง', 'มองโลก', 'เสร็จ', 'เพลิน', 'ประทับใจ', 'เปิดเผย', 'นิ่ง', 'ล้ม', 'หัว', 'เชื่อมั่น', 'มากกว่า', 'พราก', 'จอมปลอม', 'พังยับเยิน', 'สำนึก', 'ไมตรี', 'ซ้ำร้าย', 'ริดรอน', 'ชำระ', 'หล่อ', 'เเม่ง', 'ความสามัคคี', 'กุศล', 'หมั่น', 'ง่าย', 'จากไป', 'ใจ', 'กำ', 'น่าสน', 'รู้', 'มิตรภาพ', 'คลี่คลาย', 'จม', 'กบฏ', 'เจ็บแสบ', 'ดวงดี', 'เพื่อนสนิท', 'ย่อท้อ', 'เสียความรู้สึก', 'ผู้ชาย', 'สะดวก', 'สานต่อ', 'ประชาธิปไตย', 'เตก', 'เวลา', 'เจ้าชู้', 'ความเหลื่อมล้ำ', 'ฟุ่มเฟือย', 'ลม', 'พอเพียง', 'ก้อนหิน', 'สิ่งแวดล้อม', 'มะเร็ง', 'ความหวังดี', 'หกล้ม', 'ว่ะ'

In [101]:
#ทำ Set สำหรับตัว Label

label_set = set()

for a_list in new_word_list:
    if a_list[1] in label_set:
        pass
    else:
        label_set.add(a_list[1])


print(label_set)

{1, 2, 3, 4, 5}


In [102]:
print(len(word_set))
print(len(label_set))

1457
5


In [103]:
#อันนี้ลองแปลงคำเป็นตัวเลขดูเฉยๆ
#หลักการที่จะทำคือ แปลงคำเป็นตัวเลขตั้งแต่ 0 ขึ้นไปแล้วเก็บไว้ใน Numpy Array แล้วเอา Array มาแปลงเป็น Tensor อีกทีนึง

sentence_idx = np.linspace(0,len(word_set_to_list),len(word_set_to_list),False)

sentence_idx[0]

0.0

In [104]:
#แปลงคำเป็นตัวเลข โดยแต่ละคำจะมีเลขของตัวเองกำกับไว้

a = 0
link_list_1 = []

while a < len(sentence_idx):
    inner_list = []
    inner_list.append(word_set_to_list[a])
    inner_list.append(sentence_idx[a])
    link_list_1.append(inner_list)
    a += 1

print(link_list_1)

, ['ชาวบ้าน', 400.0], ['เจ็บ', 401.0], ['ลำพัง', 402.0], ['หวั่นไหว', 403.0], ['แบน', 404.0], ['ขัดแข้งขัดขา', 405.0], ['แสบแก้วหู', 406.0], ['ดี', 407.0], ['ชิม', 408.0], ['ปรับปรุง', 409.0], ['สละสิทธิ์', 410.0], ['เฮงซวย', 411.0], ['แฮก', 412.0], ['แข็งแกร่ง', 413.0], ['หลบ', 414.0], ['ความมั่นใจ', 415.0], ['ป่า', 416.0], ['พัฒนา', 417.0], ['เกียจคร้าน', 418.0], ['ดีใจ', 419.0], ['ต่อไป', 420.0], ['กอด', 421.0], ['งี่เง่า', 422.0], ['น่ากิน', 423.0], ['ทุกที่', 424.0], ['สะอาด', 425.0], ['กดดัน', 426.0], ['แย่จัง', 427.0], ['อันนี้', 428.0], ['เสน่ห์', 429.0], ['แรงบันดาลใจ', 430.0], ['สุดท้าย', 431.0], ['ระเบิด', 432.0], ['เหน็บแนม', 433.0], ['ความคลั่ง', 434.0], ['หมดแรง', 435.0], ['มี', 436.0], ['ผีห่า', 437.0], ['จุดหมาย', 438.0], ['ตามระเบียบ', 439.0], ['เย', 440.0], ['อุบัติเหตุ', 441.0], ['ดราม่า', 442.0], ['ตัวอย่าง', 443.0], ['พิเศษ', 444.0], ['โอกาส', 445.0], ['บริจาค', 446.0], ['กห', 447.0], ['ใจเร็ว', 448.0], ['ถ่ายรูป', 449.0], ['หลงทาง', 450.0], ['ทำลาย', 451.0], ['ยิ่

In [105]:
#เปลี่ยน List ที่ได้ให้เป็น Dict เตรียมสำหรับ list of keyword

new_dict = dict(link_list_1)

print(new_dict)

 'เจ้': 269.0, 'ร': 270.0, 'ปราณี': 271.0, 'ปฎิบัติ': 272.0, 'เจน': 273.0, 'ไกล': 274.0, 'บริสุทธิ์ใจ': 275.0, 'ยากลำบาก': 276.0, 'เหนื่อย': 277.0, 'ข่มขืน': 278.0, 'เขา': 279.0, 'สังเวย': 280.0, 'ทาง': 281.0, 'เกิดขึ้น': 282.0, 'วัด': 283.0, 'โจมตี': 284.0, 'ควย': 285.0, 'ตาม': 286.0, 'ตระหนัก': 287.0, 'โดดเดี่ยว': 288.0, 'ผี': 289.0, 'น่านับถือ': 290.0, 'สัตว์เดรัจฉาน': 291.0, 'ตามใจ': 292.0, 'ความตาย': 293.0, 'ทุ่มเท': 294.0, 'อุบาท': 295.0, 'มุข': 296.0, 'แทบ': 297.0, 'สนใจ': 298.0, 'ไว้อาลัย': 299.0, 'น้อย': 300.0, 'สลาย': 301.0, 'สน': 302.0, 'สม': 303.0, 'ภาวะ': 304.0, 'หมกมุ่น': 305.0, 'ยอม': 306.0, 'เดินสาย': 307.0, 'ลุกโชน': 308.0, 'นาน': 309.0, 'แถม': 310.0, 'นิยาม': 311.0, 'ปัญหา': 312.0, 'สภาวะ': 313.0, 'ความผิด': 314.0, 'ตนเอง': 315.0, 'โม': 316.0, 'เรียน': 317.0, 'ทุกครั้งที่': 318.0, 'ขยะแขยง': 319.0, 'ทางใจ': 320.0, 'ยา': 321.0, 'ดีมาก': 322.0, 'เข': 323.0, 'ไร้ค่า': 324.0, 'รู้ตัว': 325.0, 'ศักดิ์ศรี': 326.0, 'อิจฉา': 327.0, 'ไม่': 328.0, 'ตัดสินคดี': 329.0, 'คุ้น': 33

In [106]:
#เริ่มทำการเปลี่ยนคำเป็น Tensor

new_list_2 = []

for a_list in new_word_list:
    list_of_word_num = []
    for a in a_list[0]:
        b = new_dict[a]
        list_of_word_num.append(b)
    list_of_word_tensor = torch.LongTensor(list_of_word_num)
    list_combine = []
    list_combine.append(a_list[1])
    list_combine.append(list_of_word_tensor)
    new_list_2.append(list_combine)

print(new_list_2)

)], [2, tensor([1358, 1268])], [2, tensor([1401,  649])], [2, tensor([1401])], [2, tensor([1358,  878, 1268])], [2, tensor([298])], [2, tensor([1358])], [3, tensor([1334])], [3, tensor([53])], [3, tensor([610])], [3, tensor([1329])], [3, tensor([1100])], [3, tensor([186])], [3, tensor([723])], [3, tensor([242])], [3, tensor([1060])], [3, tensor([252])], [3, tensor([1308])], [3, tensor([287])], [3, tensor([1168])], [3, tensor([505])], [3, tensor([132])], [3, tensor([248])], [3, tensor([756])], [3, tensor([225])], [3, tensor([743])], [3, tensor([817])], [3, tensor([1035])], [3, tensor([151])], [3, tensor([1010])], [3, tensor([317])], [3, tensor([1444])], [3, tensor([334])], [3, tensor([1021])], [3, tensor([400])], [3, tensor([1377])], [3, tensor([501])], [3, tensor([31])], [3, tensor([601])], [3, tensor([944])], [3, tensor([920])], [3, tensor([1166])], [3, tensor([458])], [3, tensor([1273])], [3, tensor([1432])], [3, tensor([1083])], [3, tensor([602])], [3, tensor([350])], [3, tensor([54

In [107]:
#ทำการ Split Train, Test Data

train_data, test_data = train_test_split(new_list_2, test_size=0.3, random_state=42)

test_data[15]

[4, tensor([278])]

In [108]:
#Define Model
#Reference: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [159]:
# Set hyperparemeter
VOCAB_SIZE = 200000
EMBED_DIM = 32
NUN_CLASS = len(new_list_2)
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)
BATCH_SIZE = 16


In [110]:
#Define Batch size

def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [111]:
#เขียน Function สำหรับ Train และ Test

from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text, offsets, cls
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text, offsets, cls
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [112]:
#Train Model 50 รอบ ผลยัง Overfit อยู่

import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 50
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_data, [train_len, len(train_data) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.1115(train)	|	Acc: 47.8%(train)
	Loss: 0.0491(valid)	|	Acc: 48.8%(valid)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0335(train)	|	Acc: 82.4%(train)
	Loss: 0.0657(valid)	|	Acc: 62.8%(valid)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0112(train)	|	Acc: 95.8%(train)
	Loss: 0.0486(valid)	|	Acc: 62.8%(valid)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0073(train)	|	Acc: 98.4%(train)
	Loss: 0.0418(valid)	|	Acc: 65.1%(valid)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0051(train)	|	Acc: 98.4%(train)
	Loss: 0.0429(valid)	|	Acc: 65.1%(valid)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0048(train)	|	Acc: 98.8%(train)
	Loss: 0.0404(valid)	|	Acc: 67.4%(valid)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0040(train)	|	Acc: 99.1%(train)
	Loss: 0.0436(valid)	|	Acc: 65.1%(valid)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0034(train)	|	Acc: 99.3%(train)
	Loss: 0.0442(valid)	|	Acc: 67.4%(valid)
Epoch: 9  | time

In [113]:
#ลอง Test dataset ผลยังไม่ดี Overfit

print('Checking the results of test dataset...')
test_loss, test_acc = test(test_data)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0216(test)	|	Acc: 60.5%(test)


In [115]:
label_dataset = {
    1 : "Potential",
    2 : "High Potential",
    3 : "Natural",
    4 : "Negative",
    5 : "High Negative",
}

In [139]:
# ถ้าจะลองกับข้อมูลอื่นต้องแปลงให้มันเป็น Tensor ก่อนที่จะเอาไปใส่ Model

text = "ไม่"

text_tokenize = word_tokenize(text)
text_list = []
for word in text_tokenize:
    number = new_dict[word]
    text_list.append(number)
text_tensor = torch.LongTensor(text_list)

print(text_tensor)

tensor([328])


In [155]:
#รวม แปลงเป็น Tensor และใส่ Model ไว้ใน Function เดียวกัน

def predict(text):
    text_tokenize = word_tokenize(text)
    text_list = []
    for word in text_tokenize:
        number = new_dict[word]
        text_list.append(number)
    text_tensor = torch.LongTensor(text_list)
    result = model(text_tensor,torch.tensor([0]))

    return label_dataset[result.argmax(1).item()]

In [158]:
#ทดสอบ Function สามารถใช่บอกสถานะได้

predict("น่าสนใจมาก")

'Potential'